Plantilla

In [ ]:
import pandas as pd

#ruta = "https://raw.githubusercontent.com/VillaltaE/netflix_churn/refs/heads/main/netflix_customer_churn.csv"
ruta= "https://raw.githubusercontent.com/VillaltaE/netflix_churn/51eb12e70420adbd3a0a8ae243dc9e70dac78a68/df_model.csv"
df = pd.read_csv(ruta)

model_cols = [
    "subscription_type",
    "watch_hours",
    "last_login_days",
    "monthly_fee",
    "number_of_profiles",
    "avg_watch_time_per_day"
]

X = df[model_cols]
y = df["churned"]

X.head(), y.value_counts(normalize=True)


(  subscription_type  watch_hours  last_login_days  monthly_fee  \
 0             Basic        14.73               29         8.99   
 1          Standard         0.70               19        13.99   
 2          Standard        16.32               10        13.99   
 3           Premium         4.51               12        17.99   
 4          Standard         1.89               13        13.99   
 
    number_of_profiles  avg_watch_time_per_day  
 0                   1                    0.49  
 1                   5                    0.03  
 2                   2                    1.48  
 3                   2                    0.35  
 4                   2                    0.13  ,
 churned
 1    0.503
 0    0.497
 Name: proportion, dtype: float64)

Importar lo necesario

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report



Definir columnas por tipo



In [ ]:
categorical_cols = ["subscription_type"]
numeric_cols = [
    "watch_hours",
    "last_login_days",
    "monthly_fee",
    "number_of_profiles",
    "avg_watch_time_per_day"
]


Crear el preprocesador

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numeric_cols)
    ]
)


Train/Test split (80/20)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape


((4000, 6), (1000, 6))

Armar el pipeline

In [ ]:
clf = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", LogisticRegression(max_iter=1000))
])


Entrenar

In [ ]:
clf.fit(X_train, y_train)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['subscription_type']),
                                                 ('num', 'passthrough',
                                                  ['watch_hours',
                                                   'last_login_days',
                                                   'monthly_fee',
                                                   'number_of_profiles',
                                                   'avg_watch_time_per_day'])])),
                ('model', LogisticRegression(max_iter=1000))])

Evaluar

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.90      0.86      0.88       497
           1       0.87      0.91      0.89       503

    accuracy                           0.88      1000
   macro avg       0.88      0.88      0.88      1000
weighted avg       0.88      0.88      0.88      1000



Probabilidades / API

In [ ]:
y_proba = clf.predict_proba(X_test)[:, 1]
y_proba[:10]


array([1.79941400e-04, 9.85873650e-01, 8.95529186e-01, 9.90071794e-01,
       9.63331076e-01, 9.87247723e-01, 9.63685583e-01, 9.99774669e-01,
       5.33376357e-01, 8.87277884e-01])

Función de predicción

In [ ]:
def predict_cliente(input_dict):
    import pandas as pd

    df_input = pd.DataFrame([input_dict])

    proba = clf.predict_proba(df_input)[0, 1]
    pred = "Va a cancelar" if proba >= 0.5 else "Va a continuar"

    return {
        "prevision": pred,
        "probabilidad": round(float(proba), 3)
    }


Probar con un cliente / Manual

In [ ]:
cliente_ejemplo = {
    "subscription_type": "Basic",
    "watch_hours": 1.2,
    "last_login_days": 40,
    "monthly_fee": 8.99,
    "number_of_profiles": 1,
    "avg_watch_time_per_day": 0.2
}

predict_cliente(cliente_ejemplo)


{'prevision': 'Va a cancelar', 'probabilidad': 0.999}